In [24]:
from utils.notebook_helper import setup_notebook_environment
setup_notebook_environment()

'Environment ready for experiments'

In [25]:
from llm.router import LLMRouter

client = LLMRouter(model="mistral")


In [26]:
print(client.ask("Hello, who are you?"))

 Hello! I'm an assistant designed to help answer questions, provide information, and assist with various tasks. How can I help you today?


# Making Curl Requests from Jupyter

There are several ways to make curl requests from Jupyter notebooks:

In [3]:
!curl -X GET http://0.0.0.0:8000/health



curl: (7) Failed to connect to 0.0.0.0 port 8000 after 0 ms: Couldn't connect to server


In [13]:
import requests
chat_data = {
    "message": "Hello from Jupyter arte yiou working!",
}
response = requests.post('http://langchain_api:8000/health',  json=chat_data)
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")

Status Code: 405
Response: {'detail': 'Method Not Allowed'}


In [8]:
# Method 3: Using requests library (recommended)
import requests
chat_data = {
    "message": "Hello from Jupyter!",
}
# GET request
response = requests.post('http://langchain_api:8000/chat', json=chat_data)
print(f"Status Code: {response.status_code}")
print(f"Response: {response.json()}")

Status Code: 200
Response: {'response': 'Hello! Greetings from Jupyter! How can I assist you today?', 'provider': 'openai', 'model': 'gpt-4.1-nano'}


In [ ]:
# Method 4: POST request with JSON data
import requests

# Test the chat endpoint
chat_data = {
    "message": "Hello from Jupyter!",
    "model": "gpt4"
}

response = requests.post(
    'http://0.0.0.0:8000/chat',
    json=chat_data
)

print(f"Status Code: {response.status_code}")
if response.status_code == 200:
    print(f"Response: {response.json()}")
else:
    print(f"Error: {response.text}")

In [ ]:
# Method 5: Equivalent curl command as shell
!curl -X POST http://0.0.0.0:8000/chat \
  -H "Content-Type: application/json" \
  -d '{"message": "Hello from curl!", "model": "gpt3"}'

In [ ]:
# Method 6: Check available providers
response = requests.get('http://0.0.0.0:8000/providers')
print(f"Available providers: {response.json()}")